In [4]:
%pip install openai
%pip install pandas

Note: you may need to restart the kernel to use updated packages.
  Using cached pandas-2.2.2-cp312-cp312-macosx_11_0_arm64.whl.metadata (19 kB)
  Using cached numpy-2.0.0-cp312-cp312-macosx_14_0_arm64.whl.metadata (60 kB)
  Using cached pytz-2024.1-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2024.1-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached pandas-2.2.2-cp312-cp312-macosx_11_0_arm64.whl (11.3 MB)
Using cached numpy-2.0.0-cp312-cp312-macosx_14_0_arm64.whl (5.0 MB)
Using cached pytz-2024.1-py2.py3-none-any.whl (505 kB)
Using cached tzdata-2024.1-py2.py3-none-any.whl (345 kB)


In [1]:
from openai import OpenAI
import os
import json
def request_hint(word):
    API_KEY = "sk-jCsvhJjSF0Jaq63NrfZJnvFsvNyXcO6WtuuhIAs70cgR77jo" #

    client = OpenAI(
        api_key=API_KEY,  
        base_url="https://api.opentyphoon.ai/v1",
    )

    stream = client.chat.completions.create(
        model="typhoon-v1.5x-70b-instruct",
        messages=[
            {
                "role": "system",
                "content": 
                """
                คุณเป็นเจ้าหน้าที่ตรวจสอบคอมเม้นในบริษัทโซเชียลมีเดีย.
                มีหน้าที่ตรวจสอบข้อความและระบุเจตนาหรืออารมณ์ในแต่ละข้อความ,พิจารณาอิโมจิ.
                โดยที่เจตนาบวกแทนด้วยเลข 0, ไม่มีเจตนาหรืออารมณ์แทนด้วยเลข 1,เจตนาลบแทนด้วยเลข 2, และเจตนาถามแทนด้วยเลข 3.
                ให้คุณวิเคราะห์เจตนาหรืออารมณ์ข้อความของ ผู้ใช้งาน และระบุตัวเลขเจตนาตามที่กำหนดไว้.
                โดยคิดอย่างเป็นขั้นตอน.

                ตัวอย่างการวิเคราะห์ข้อความ เจตนา และเหตุผล:
                - เปิดศักราชใหม่! นายกฯ แถลงข่าวก่อนการแข่งขันศึก #ช้างเอฟเอคัพ นัดชิงชนะเลิศ => 1 เพราะคอมเม้นนี้สื่ออารมณ์เป็นกลาง
                - (emoji:smiling_face_with_heart-eyes::smiling_face_with_heart-eyes:) => 0 เพราะคอมเม้นนี้สื่อไปในทางรัก
                - บัตรสมาชิกลดได้อีกไหมคับ => 3 เพราะคอมเม้นนี้สื่อไปในการตั้งคำถาม
                - 张丽霞 อ้อออ ผ้าอนามัยแบบสอดอ่ะนะ กุกลัวมาก 5555555 => 2  เพราะคอมเม้นนี้สื่ออารมณ์ความกังวน กลัว
             

                """
            },
            {
                "role": "user",
               # "content": f"ผมอยากฝึกพูดคำว่า {word}"
                "content": f"ลูกค้าคอมเม้นบอกว่า '{word}' มีเจตนาหรืออารมณ์ ตรงกับเลขอะไร (ตอบเฉพาะตัวเลข)"

            }
        ],
        max_tokens=200,
        temperature=0.0,
        top_p=1,
        stream=True,
    )
    output = ""
    for chunk in stream:
        if hasattr(chunk, 'choices') and len(chunk.choices) > 0:
            choice = chunk.choices[0]
            if hasattr(choice, 'delta') and hasattr(choice.delta, 'content'):
                print(choice.delta.content,end="")
                output+=choice.delta.content
 
    return output

ModuleNotFoundError: No module named 'openai'

### RAG

In [3]:
import pandas as pd
from openai import OpenAI

# Load the CSV data
data_path = './train.csv'
df = pd.read_csv(data_path)
df = df.tail(20)

# Initialize OpenAI client
API_KEY = "sk-jCsvhJjSF0Jaq63NrfZJnvFsvNyXcO6WtuuhIAs70cgR77jo"
client = OpenAI(api_key=API_KEY, base_url="https://api.opentyphoon.ai/v1")

# Define the RAG function
def classify_sentiment(text):
    # Define the retrieval and generation prompt
    prompt = f"""
    คุณเป็นเจ้าหน้าที่ตรวจสอบคอมเม้นในบริษัทโซเชียลมีเดีย.
    มีหน้าที่ตรวจสอบข้อความและระบุเจตนาหรืออารมณ์ในแต่ละข้อความ,พิจารณาอิโมจิ.
    โดยที่เจตนาบวกแทนด้วยเลข 0, ไม่มีเจตนาหรืออารมณ์แทนด้วยเลข 1,เจตนาลบแทนด้วยเลข 2, และเจตนาถามแทนด้วยเลข 3.
    ให้คุณวิเคราะห์เจตนาหรืออารมณ์ข้อความของ ผู้ใช้งาน และระบุตัวเลขเจตนาตามที่กำหนดไว้.
    โดยคิดอย่างเป็นขั้นตอน.

    ตัวอย่างการวิเคราะห์ข้อความ เจตนา และเหตุผล:
    - เปิดศักราชใหม่! นายกฯ แถลงข่าวก่อนการแข่งขันศึก #ช้างเอฟเอคัพ นัดชิงชนะเลิศ => 1 เพราะคอมเม้นนี้สื่ออารมณ์เป็นกลาง
    - (emoji:smiling_face_with_heart-eyes::smiling_face_with_heart-eyes:) => 0 เพราะคอมเม้นนี้สื่อไปในทางรัก
    - บัตรสมาชิกลดได้อีกไหมคับ => 3 เพราะคอมเม้นนี้สื่อไปในการตั้งคำถาม
    - 张丽霞 อ้อออ ผ้าอนามัยแบบสอดอ่ะนะ กุกลัวมาก 5555555 => 2  เพราะคอมเม้นนี้สื่ออารมณ์ความกังวน กลัว

    ลูกค้าคอมเม้นบอกว่า '{text}' มีเจตนาหรืออารมณ์ ตรงกับเลขอะไร (ตอบเฉพาะตัวเลข)
    """

    # Make a request to the OpenAI API
    response = client.chat.completions.create(
        model="typhoon-v1.5x-70b-instruct",
        messages=[
            {
                "role": "system",
                "content": prompt
            },
            {
                "role": "user",
                "content": text
            }
        ],
        max_tokens=200,
        temperature=0.0,
        top_p=1,
        stream=False
    )

    # Extract the classification result from the response
    output = response.choices[0].message.content.strip()
    return output

# Apply the RAG model to classify sentiments for each text in the dataset
df['predicted_category'] = df['texts'].apply(classify_sentiment)

# Save the results
output_path = './predicted_train.csv'
df.to_csv(output_path, index=False)
print(f"Results saved to {output_path}")


2

'2'